# En búsqueda de Alta Cohesión y Bajo Acoplamiento

Material proporcionado por el Prof. Daniel Salazar Mora.

## Alta vs Baja Cohesión

**Alta cohesión:** Los elementos de un módulo están alineados y tienen un propósito claro.

**Baja cohesión:** Los elementos de un módulo realizan tareas variadas y no relacionadas.

### Ejemplo de Baja Cohesión (mala)

In [1]:
class OrderSystem:
    def __init__(self):
        self.orders = []

    def add_order(self, order):
        self.orders.append(order)

    def process_orders(self):
        for order in self.orders:
            print(f"Procesando orden: {order}")

    def calculate_total(self):
        total = sum(order['amount'] for order in self.orders)
        print(f"Total de órdenes: {total}")

    def send_email_confirmation(self):
        print("Enviando confirmación por correo...")

### Ejemplo de Alta Cohesión (deseada)

In [2]:
class Order:
    def __init__(self, order_id, amount):
        self.order_id = order_id
        self.amount = amount

class OrderManager:
    def __init__(self):
        self.orders = []

    def add_order(self, order):
        self.orders.append(order)

    def process_orders(self):
        for order in self.orders:
            print(f"Procesando orden: {order.order_id}")

class PaymentProcessor:
    @staticmethod
    def calculate_total(orders):
        total = sum(order.amount for order in orders)
        print(f"Total de órdenes: {total}")

class NotificationService:
    @staticmethod
    def send_email():
        print("Enviando confirmación por correo...")

### ¿Diferencias?

Al utilizar alta cohesion, es más fácil de mantener y extender el código. Por ejemplo, podemos realizar cambios en el NotificationService y tenemos mucha confianza en que no habrán cambios en las otras funcionalidades por consecuencia a esta.

Ademas de que es mas natural de invocar y utilizar el código de esta forma.

In [3]:
order_manager = OrderManager()

order1 = Order(1, 100)
order2 = Order(2, 200)

order_manager.add_order(order1)
order_manager.add_order(order2)

order_manager.process_orders()
PaymentProcessor.calculate_total(order_manager.orders)
NotificationService.send_email()

Procesando orden: 1
Procesando orden: 2
Total de órdenes: 300
Enviando confirmación por correo...


## Alto vs Bajo Acoplamiento

**Bajo acoplamiento:** Los módulos dependen poco entre sí y se comunican a través de interfaces bien definidas.

**Alto acoplamiento:** Los módulos están muy interconectados, lo que hace difícil modificar uno sin afectar a los demás.

### Alto acoplamiento (malo)

In [5]:
class Database:
    def save_data(self, data):
        print(f"Guardando en base de datos: {data}")

class Report:
    def __init__(self):
        self.database = Database()  # Dependencia directa

    def generate_report(self):
        print("Generando reporte...")
        self.database.save_data("Reporte generado")  # Llamada directa

# Uso en el programa
report = Report()
report.generate_report()

Generando reporte...
Guardando en base de datos: Reporte generado


### Bajo acoplamiento (deseado)

In [6]:
from abc import ABC, abstractmethod

# Definir una interfaz de almacenamiento
class Storage(ABC):
    @abstractmethod
    def save_data(self, data):
        pass

# Implementación de almacenamiento en base de datos
class DatabaseStorage(Storage):
    def save_data(self, data):
        print(f"Guardando en base de datos: {data}")

# Implementación de almacenamiento en archivo
class FileStorage(Storage):
    def save_data(self, data):
        print(f"Guardando en un archivo: {data}")

# Report ya no depende de una implementación específica
class Report:
    def __init__(self, storage: Storage):
        self.storage = storage  # Inyección de dependencia

    def generate_report(self):
        print("Generando reporte...")
        self.storage.save_data("Reporte generado")  # Uso de la interfaz

# Uso en el programa
database_storage = DatabaseStorage()
file_storage = FileStorage()

report1 = Report(database_storage)  # Usando base de datos
report1.generate_report()

report2 = Report(file_storage)  # Usando archivos
report2.generate_report()

Generando reporte...
Guardando en base de datos: Reporte generado
Generando reporte...
Guardando en un archivo: Reporte generado


### ¿Diferencias?

Al manejar bajo acoplamiento, evitamos dependencias directas que pueden requerir muchos cambios en un futuro. En este caso, si en un futuro quisieramos agregar almacenamiento en la nube, entonces lo unico que tendriamos que hacer es crear una nueva implementación de Storage e invocarla en Report. Ni siquiera tendríamos que cambiar el código ya escrito!